In [1]:
from ast import literal_eval
import pandas as pd
import os
import cv2
import imagesize
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [2]:
from src.full_model.evaluate_bbox_variations.custom_dataset_bbox_variations import CustomDataset

In [3]:
IMAGE_INPUT_SIZE = 512

In [4]:
path_to_partial_test_set = "/u/home/tanida/datasets/dataset-with-reference-reports-partial-1000/test-1000.csv"

In [5]:
def get_test_set_as_df():
    def compute_bbox_widths_heights(row):
        bbox_coordinates_single_image = row["bbox_coordinates"]
        widths_heights = []
        for bbox_coords in bbox_coordinates_single_image:
            x1, y1, x2, y2 = bbox_coords
            width = x2 - x1
            height = y2 - y1
            widths_heights.append([width, height])

        return widths_heights

    def retrieve_image_widths_heights(row):
        mimic_image_file_path = row["mimic_image_file_path"]
        width, height = imagesize.get(mimic_image_file_path)
        return [width, height]

    usecols = [
        "mimic_image_file_path",
        "bbox_coordinates",
        "bbox_labels",
        "bbox_phrases",
        "bbox_phrase_exists",
    ]

    # all of the columns below are stored as strings in the csv_file
    # however, as they are actually lists, we apply the literal_eval func to convert them to lists
    converters = {
        "bbox_coordinates": literal_eval,
        "bbox_labels": literal_eval,
        "bbox_phrases": literal_eval,
        "bbox_phrase_exists": literal_eval,
    }

    test_set_as_df = pd.read_csv(path_to_partial_test_set, usecols=usecols, converters=converters)

    # add new columns that contain the bbox_widths_heights (List[List[int]] with len(outer_list)=29 and len(inner_list) = 2)
    # and image_width_height (List[int] of len 2)
    test_set_as_df["bbox_widths_heights"] = test_set_as_df.apply(lambda row: compute_bbox_widths_heights(row), axis=1)
    test_set_as_df["image_width_height"] = test_set_as_df.apply(lambda row: retrieve_image_widths_heights(row), axis=1)

    return test_set_as_df

In [6]:
dataset_as_df = get_test_set_as_df()[:3]

In [7]:
dataset_as_df.head()

,mimic_image_file_path,bbox_coordinates,bbox_labels,bbox_phrases,bbox_phrase_exists,bbox_widths_heights,image_width_height
0,/u/home/tanida/datasets/mimic-cxr-jpg/files/p1...,"[[327, 231, 1200, 2114], [477, 300, 1200, 968]...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[There is no focal consolidation, pleural effu...","[True, False, False, True, False, False, True,...","[[873, 1883], [723, 668], [751, 423], [819, 72...","[2544, 3056]"
1,/u/home/tanida/datasets/mimic-cxr-jpg/files/p1...,"[[300, 382, 1227, 2332], [477, 436, 1227, 1118...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",[Pulmonary vasculature is normal. Lungs are cl...,"[True, False, False, False, True, False, True,...","[[927, 1950], [750, 682], [778, 437], [859, 77...","[2544, 3056]"
2,/u/home/tanida/datasets/mimic-cxr-jpg/files/p1...,"[[229, 652, 1171, 2330], [386, 676, 1086, 1135...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",[No acute intrathoracic process. There is no f...,"[True, False, False, False, False, True, True,...","[[942, 1678], [700, 459], [870, 434], [942, 76...","[2539, 2705]"


In [8]:
def vary_bbox_coords_position(row):
    def check_coordinate(coord, dimension):
        """Make sure that new coordinate is still within the image."""
        if coord < 0:
            return 0
        elif coord > dimension:
            return dimension
        else:
            return coord

    bbox_coords_single_image = row["bbox_coordinates"]  # List[List[int]] of shape 29 x 4
    bbox_widths_heights_single_image = row["bbox_widths_heights"]  # List[List[int]] of shape 29 x 2
    relative_position_variation_bboxes = row["relative_position_variations"]  # List[List[float]] of shape 29 x 2
    image_width, image_height = row["image_width_height"]  # two integers

    # to store the new bbox coordinates after they have been varied
    varied_bbox_coords_single_image = []

    for bbox_coords, bbox_width_height, relative_position_variations in zip(bbox_coords_single_image, bbox_widths_heights_single_image, relative_position_variation_bboxes):
        x1, y1, x2, y2 = bbox_coords
        bbox_width, bbox_height = bbox_width_height
        x_rel, y_rel = relative_position_variations

        # if e.g. x_rel = 0.5 and bbox_width = 100, then x_var = 50
        x_var = int(bbox_width * x_rel)
        y_var = int(bbox_height * y_rel)

        x1 += x_var
        x2 += x_var
        y1 += y_var
        y2 += y_var

        x1 = check_coordinate(x1, image_width)
        x2 = check_coordinate(x2, image_width)
        y1 = check_coordinate(y1, image_height)
        y2 = check_coordinate(y2, image_height)

        varied_bbox_coords_single_image.append([x1, y1, x2, y2])

    return varied_bbox_coords_single_image

In [9]:
num_images = len(dataset_as_df)
mean = 0
std = 0.1

relative_position_variations = np.random.normal(mean, std, size=(num_images, 29, 2))
dataset_as_df["relative_position_variations"] = relative_position_variations.tolist()
dataset_as_df["bbox_coordinates_varied"] = dataset_as_df.apply(lambda row: vary_bbox_coords_position(row), axis=1)

In [27]:
img_path = dataset_as_df.iloc[0]["mimic_image_file_path"]
image_cv2 = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
image_cv2.shape

(3056, 2544)

In [37]:
test_transforms = A.Compose(
        [
            A.Resize(height=700, width=300, interpolation=cv2.INTER_AREA),
            # A.PadIfNeeded(min_height=IMAGE_INPUT_SIZE, min_width=IMAGE_INPUT_SIZE, border_mode=cv2.BORDER_CONSTANT),
            A.Normalize(mean=0.471, std=0.302),
            ToTensorV2(),
        ]
    )

transformed_img_cv2 = test_transforms(image=image_cv2)["image"]
transformed_img_cv2.shape

torch.Size([1, 700, 300])

In [10]:
dataset_as_df.head()

,mimic_image_file_path,bbox_coordinates,bbox_labels,bbox_phrases,bbox_phrase_exists,bbox_widths_heights,image_width_height,relative_position_variations,bbox_coordinates_varied
0,/u/home/tanida/datasets/mimic-cxr-jpg/files/p1...,"[[327, 231, 1200, 2114], [477, 300, 1200, 968]...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[There is no focal consolidation, pleural effu...","[True, False, False, True, False, False, True,...","[[873, 1883], [723, 668], [751, 423], [819, 72...","[2544, 3056]","[[0.04699671459135207, 0.09165250484250043], [...","[[368, 403, 1241, 2286], [462, 214, 1185, 882]..."
1,/u/home/tanida/datasets/mimic-cxr-jpg/files/p1...,"[[300, 382, 1227, 2332], [477, 436, 1227, 1118...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",[Pulmonary vasculature is normal. Lungs are cl...,"[True, False, False, False, True, False, True,...","[[927, 1950], [750, 682], [778, 437], [859, 77...","[2544, 3056]","[[-0.14522310411740774, -0.10162471564409498],...","[[166, 184, 1093, 2134], [516, 421, 1266, 1103..."
2,/u/home/tanida/datasets/mimic-cxr-jpg/files/p1...,"[[229, 652, 1171, 2330], [386, 676, 1086, 1135...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",[No acute intrathoracic process. There is no f...,"[True, False, False, False, False, True, True,...","[[942, 1678], [700, 459], [870, 434], [942, 76...","[2539, 2705]","[[-0.032675926785905994, -0.029834814364771874...","[[199, 602, 1141, 2280], [289, 685, 989, 1144]..."


In [11]:
def get_transforms():
    # see compute_mean_std_dataset.py in src/dataset
    mean = 0.471
    std = 0.302

    # don't apply data augmentations to test set
    test_transforms = A.Compose(
        [
            A.LongestMaxSize(max_size=IMAGE_INPUT_SIZE, interpolation=cv2.INTER_AREA),
            A.PadIfNeeded(min_height=IMAGE_INPUT_SIZE, min_width=IMAGE_INPUT_SIZE, border_mode=cv2.BORDER_CONSTANT),
            A.Normalize(mean=mean, std=std),
            ToTensorV2(),
        ],
        bbox_params=A.BboxParams(format="pascal_voc", label_fields=["class_labels"]),
    )

    return test_transforms

transforms = get_transforms()

In [12]:
dataset = CustomDataset(dataset_as_df, transforms, log=None)

In [14]:
image = sample_1["image"]
image.shape

torch.Size([1, 512, 512])

In [20]:
four_bbox_coords = sample_1["bbox_coordinates"][0]
x1, y1, x2, y2 = four_bbox_coords
x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
print(x1, y1, x2, y2)

104 67 250 382


In [24]:
width = x2 - x1
height = y2 - y1
print(width)
print(height)

146
315


In [25]:
from torchvision.transforms.functional import crop

In [26]:
image_bbox_new = crop(image, top=y1, left=x1, height=height, width=width)
image_bbox_new.shape

torch.Size([1, 315, 146])

In [35]:
image_bbox = image[:, y1:y2, x1:x2]
image_bbox.shape

torch.Size([1, 315, 146])

In [13]:
sample_1 = dataset[0]
sample_1

{'image': tensor([[[-1.5596, -1.5596, -1.5596,  ..., -1.5596, -1.5596, -1.5596],
          [-1.5596, -1.5596, -1.5596,  ..., -1.5596, -1.5596, -1.5596],
          [-1.5596, -1.5596, -1.5596,  ..., -1.5596, -1.5596, -1.5596],
          ...,
          [-1.5596, -1.5596, -1.5596,  ..., -1.5596, -1.5596, -1.5596],
          [-1.5596, -1.5596, -1.5596,  ..., -1.5596, -1.5596, -1.5596],
          [-1.5596, -1.5596, -1.5596,  ..., -1.5596, -1.5596, -1.5596]]]),
 'bbox_coordinates': [(104.62264150943396,
   67.51832460732984,
   250.80896226415095,
   382.9947643979058),
  (120.36320754716981,
   35.853403141361255,
   241.4316037735849,
   147.7696335078534),
  (109.81367924528303,
   156.98429319371726,
   235.57075471698113,
   227.85340314136127),
  (84.52830188679246,
   240.92146596858638,
   221.67216981132074,
   362.0523560209424),
  (174.95283018867923,
   157.82198952879583,
   241.09669811320757,
   258.3455497382199),
  (141.62971698113208,
   43.72774869109948,
   251.31132075471